In [2]:
pip install sb3-contrib
# 注意：如果安装失败，尝试关闭vpn

Could not fetch URL https://pypi.org/simple/sb3-contrib/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/sb3-contrib/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping


ERROR: Could not find a version that satisfies the requirement sb3-contrib
ERROR: No matching distribution found for sb3-contrib


In [2]:
from sb3_contrib import MaskablePPO

In [3]:
import numpy as np

In [6]:
np.array([1,2,3])

array([1, 2, 3])

In [8]:
from stable_baselines3.common.env_checker import check_env
from gym_reversi import ReversiEnv

env = ReversiEnv(player_color='black', opponent = "random", board_size=8)
# It will check your custom environment and output additional warnings if needed
check_env(env)


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:38: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:51: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


In [5]:
??make_vec_env

In [6]:
??ActionMasker

### Maskable PPO

In [2]:
# import sys
import os
# current_path = os.getcwd()
# sys.path.append(current_path)
import os
import time
import gymnasium as gym
import numpy as np

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
# from stable_baselines3 import DQN, DDPG, A2C, PPO,SAC,TD3
from stable_baselines3 import PPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

import time
from gym_reversi import ReversiEnv
from custom_feature_extractor import CustomCNN

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# env = make_atari_env("BreakoutNoFrameskip-v4", seed=0)
# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, 
# #                          seed=0
#                         )
# # Frame-stacking with 4 frames
# vec_env = VecFrameStack(vec_env, n_stack=4)

def mask_fn(env: gym.Env) -> np.ndarray:
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.valid_action_mask()

backbone= 'resnet'
# backbone= 'cnn'


board_size=8
total_timesteps=10_0000
# PolicyModel = PPO
PolicyModel = MaskablePPO
# PolicyModel = TD3
n_envs = 4

greedy_rate=0
verbose = 0

tensorboard_log = f"models/ppo_{board_size}x{board_size}_{backbone}_debug/"
if not os.path.isdir(tensorboard_log):
    os.makedirs(tensorboard_log)
model_path = os.path.join(tensorboard_log, "model")
opponent_model_path="random"
# opponent_model_path=os.path.join(tensorboard_log, "opponent_model")


if opponent_model_path != "random":
    opponent_model = PolicyModel.load(opponent_model_path)
else:
    opponent_model = "random"

env = ReversiEnv(opponent=opponent_model, is_train=True, board_size=board_size, n_channels=4,
                 greedy_rate=greedy_rate, verbose=verbose)

# env = ActionMasker(env, mask_fn)  # Wrap to enable masking

vec_env = env
if n_envs > 1:
    # multi-worker training (n_envs=4 => 4 environments)
    vec_env = make_vec_env(ReversiEnv, n_envs=n_envs, seed=None,
                           env_kwargs={
                               "opponent": opponent_model,
                               "is_train": True,
                               "board_size": board_size,
                               "greedy_rate": greedy_rate,
                               "verbose": verbose},
                        )

    
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256,
                                   backbone=backbone,
                                   net_arch=[64, 64, 128],
#                                    net_arch=[32, 64, 64],
                                   kernel_size=3, 
                                   stride=1, 
                                   padding='same', 
                                   is_batch_norm=False),
    net_arch=[256, 256], 
    normalize_images=False
)
    
try:
    model = PolicyModel.load(model_path, env=vec_env)
except Exception:
    print(f"load model from self.model_path: {model_path} error")
    model = PolicyModel(MaskableActorCriticPolicy, vec_env,
                  policy_kwargs=policy_kwargs,
                  learning_rate=2.5e-4,  # learning_rate=2.5e-4,
                  ent_coef=0.01,
                  n_steps=64, # n_steps=128,
                  n_epochs=4,
                  batch_size=32, # batch_size=256,
                  gamma=0.9,
                  gae_lambda=0.9,
                  clip_range=0.2,
                  vf_coef=0.5,
                  verbose=1,
                  tensorboard_log=tensorboard_log)

t0 = time.time()
# model.learn(int(2e4))
model.learn(total_timesteps=total_timesteps)
model.save(model_path)
print(f"train time: {time.time()-t0}")

# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
# vec_env = VecFrameStack(vec_env, n_stack=4)


inplanes:4
planes:64
inplanes:64
planes:64
inplanes:64
planes:128
Logging to models/ppo_8x8_resnet_debug/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.1     |
|    ep_rew_mean     | 0.5      |
| time/              |          |
|    fps             | 206      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 256      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | 0.375       |
| time/                   |             |
|    fps                  | 141         |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.062315006 |
|    clip_fraction        | 0.335       |
|    clip_range           | 0.2         |
|  

KeyboardInterrupt: 

In [3]:
tensorboard --logdir ./reversi/models/Reversi_ppo/PPO_7/ --port=6016

8

In [ ]:
import torch
from stable_baselines3 import PPO
PolicyModel = PPO

model_path = "models/ppo_8x8_cnn/model_1480w"
# model_path = "models/model_460w"
state_dict_path = model_path + '_state_dict.pt'

sb3_model = PolicyModel.load(model_path)

torch.save(sb3_model.policy.state_dict(), state_dict_path)

In [ ]:
import torch
from model_deploy.policies import CustomCNN, ActorCriticPolicy


features_extractor_kwargs=dict(features_dim=256,
#                                net_arch=[64, 128, 256],
                               # net_arch=[64, 128, 128],
#                                net_arch=[32, 64, 128],
                               net_arch=[32, 64, 64],
                               kernel_size=3,
                               stride=1,
                               padding='same',
                               is_batch_norm=False
                               )

observation_space=(4, 8, 8)
action_space = [observation_space[1]*observation_space[2]]

policy_model = ActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)

# model_path = "models/ppo_8x8_cnn/model"
model_path = "models/ppo_8x8_cnn/model_1480w"
# model_path = "models/model_460w"

# model_path = "models/model"
state_dict_path = model_path + '_state_dict.pt'

policy_model.load_state_dict(torch.load(state_dict_path))

In [45]:
policy_model

ActorCriticPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=Tr

In [46]:
policy_model.load_state_dict(torch.load("models/ppo_8x8_cnn/model_1480w_state_dict.pt"))

<All keys matched successfully>

In [48]:
import time
import torch
from stable_baselines3 import PPO
from gym_reversi import ReversiEnv

PolicyModel = PPO

opponent_model = "random"
# opponent_model = PolicyModel.load("models/Reversi_ppo/model")

env = ReversiEnv(opponent="random", board_size=8, player_color='white', is_train=False,
                 greedy_rate=0, verbose=0)

# model = PolicyModel.load("models/Reversi_ppo/model")
# model = PolicyModel.load('models/model_240w')
# model = torch.load('models/model_240w.pth')
# model = PolicyModel.load("models/ppo_8x8_cnn/model_1500w")

policy_model.load_state_dict(torch.load("models/ppo_8x8_cnn/model_1480w_state_dict.pt"))
model = policy_model

max_round = 1000

total_round = 0
total_win = 0
total_failure = 0
total_equal = 0


t0=time.time()
obs, info = env.reset()
while total_round < max_round:
    action, _states = model.predict(obs, deterministic=True)
    action = int(action)
    obs, rewards, dones, truncated, info = env.step(action)

#     print(f"---- round:{total_round} --------")
#     print(f"action: {action}")
#     env.render("human")

    if dones:
        print(f"\n---- round:{total_round} --------")
#         env.render("human")
        obs, info = env.reset()
        total_round += 1
        if rewards > 0:
            total_win+=1
        elif rewards < 0:
            total_failure += 1
        else:
            total_equal += 1

        print(f"total_win:{total_win}, total_failure: {total_failure}, total_equal:{total_equal}\n")

# print(f"total_win:{total_win}, total_failure: {total_failure}")
print(f"train time: {time.time()-t0}")



---- round:0 --------
total_win:0, total_failure: 1, total_equal:0


---- round:1 --------
total_win:1, total_failure: 1, total_equal:0


---- round:2 --------
total_win:2, total_failure: 1, total_equal:0


---- round:3 --------
total_win:3, total_failure: 1, total_equal:0


---- round:4 --------
total_win:4, total_failure: 1, total_equal:0


---- round:5 --------
total_win:5, total_failure: 1, total_equal:0


---- round:6 --------
total_win:5, total_failure: 2, total_equal:0


---- round:7 --------
total_win:6, total_failure: 2, total_equal:0


---- round:8 --------
total_win:6, total_failure: 3, total_equal:0


---- round:9 --------
total_win:7, total_failure: 3, total_equal:0


---- round:10 --------
total_win:8, total_failure: 3, total_equal:0


---- round:11 --------
total_win:8, total_failure: 4, total_equal:0


---- round:12 --------
total_win:9, total_failure: 4, total_equal:0


---- round:13 --------
total_win:10, total_failure: 4, total_equal:0


---- round:14 --------
total


---- round:115 --------
total_win:72, total_failure: 41, total_equal:3


---- round:116 --------
total_win:73, total_failure: 41, total_equal:3


---- round:117 --------
total_win:74, total_failure: 41, total_equal:3


---- round:118 --------
total_win:75, total_failure: 41, total_equal:3


---- round:119 --------
total_win:75, total_failure: 41, total_equal:4


---- round:120 --------
total_win:76, total_failure: 41, total_equal:4


---- round:121 --------
total_win:76, total_failure: 42, total_equal:4


---- round:122 --------
total_win:76, total_failure: 43, total_equal:4


---- round:123 --------
total_win:77, total_failure: 43, total_equal:4


---- round:124 --------
total_win:77, total_failure: 43, total_equal:5


---- round:125 --------
total_win:77, total_failure: 44, total_equal:5


---- round:126 --------
total_win:77, total_failure: 45, total_equal:5


---- round:127 --------
total_win:77, total_failure: 46, total_equal:5


---- round:128 --------
total_win:78, total_failur


---- round:227 --------
total_win:136, total_failure: 83, total_equal:9


---- round:228 --------
total_win:136, total_failure: 84, total_equal:9


---- round:229 --------
total_win:137, total_failure: 84, total_equal:9


---- round:230 --------
total_win:138, total_failure: 84, total_equal:9


---- round:231 --------
total_win:139, total_failure: 84, total_equal:9


---- round:232 --------
total_win:140, total_failure: 84, total_equal:9


---- round:233 --------
total_win:141, total_failure: 84, total_equal:9


---- round:234 --------
total_win:141, total_failure: 85, total_equal:9


---- round:235 --------
total_win:142, total_failure: 85, total_equal:9


---- round:236 --------
total_win:143, total_failure: 85, total_equal:9


---- round:237 --------
total_win:144, total_failure: 85, total_equal:9


---- round:238 --------
total_win:145, total_failure: 85, total_equal:9


---- round:239 --------
total_win:145, total_failure: 86, total_equal:9


---- round:240 --------
total_win:146


---- round:337 --------
total_win:203, total_failure: 123, total_equal:12


---- round:338 --------
total_win:204, total_failure: 123, total_equal:12


---- round:339 --------
total_win:204, total_failure: 124, total_equal:12


---- round:340 --------
total_win:204, total_failure: 125, total_equal:12


---- round:341 --------
total_win:205, total_failure: 125, total_equal:12


---- round:342 --------
total_win:205, total_failure: 126, total_equal:12


---- round:343 --------
total_win:205, total_failure: 127, total_equal:12


---- round:344 --------
total_win:206, total_failure: 127, total_equal:12


---- round:345 --------
total_win:207, total_failure: 127, total_equal:12


---- round:346 --------
total_win:208, total_failure: 127, total_equal:12


---- round:347 --------
total_win:208, total_failure: 128, total_equal:12


---- round:348 --------
total_win:209, total_failure: 128, total_equal:12


---- round:349 --------
total_win:209, total_failure: 129, total_equal:12


---- round:


---- round:445 --------
total_win:263, total_failure: 166, total_equal:17


---- round:446 --------
total_win:264, total_failure: 166, total_equal:17


---- round:447 --------
total_win:265, total_failure: 166, total_equal:17


---- round:448 --------
total_win:265, total_failure: 167, total_equal:17


---- round:449 --------
total_win:266, total_failure: 167, total_equal:17


---- round:450 --------
total_win:266, total_failure: 168, total_equal:17


---- round:451 --------
total_win:267, total_failure: 168, total_equal:17


---- round:452 --------
total_win:268, total_failure: 168, total_equal:17


---- round:453 --------
total_win:268, total_failure: 169, total_equal:17


---- round:454 --------
total_win:269, total_failure: 169, total_equal:17


---- round:455 --------
total_win:269, total_failure: 170, total_equal:17


---- round:456 --------
total_win:269, total_failure: 171, total_equal:17


---- round:457 --------
total_win:270, total_failure: 171, total_equal:17


---- round:


---- round:553 --------
total_win:321, total_failure: 215, total_equal:18


---- round:554 --------
total_win:321, total_failure: 216, total_equal:18


---- round:555 --------
total_win:321, total_failure: 217, total_equal:18


---- round:556 --------
total_win:322, total_failure: 217, total_equal:18


---- round:557 --------
total_win:322, total_failure: 218, total_equal:18


---- round:558 --------
total_win:323, total_failure: 218, total_equal:18


---- round:559 --------
total_win:323, total_failure: 219, total_equal:18


---- round:560 --------
total_win:324, total_failure: 219, total_equal:18


---- round:561 --------
total_win:325, total_failure: 219, total_equal:18


---- round:562 --------
total_win:325, total_failure: 220, total_equal:18


---- round:563 --------
total_win:326, total_failure: 220, total_equal:18


---- round:564 --------
total_win:326, total_failure: 221, total_equal:18


---- round:565 --------
total_win:327, total_failure: 221, total_equal:18


---- round:


---- round:661 --------
total_win:387, total_failure: 255, total_equal:20


---- round:662 --------
total_win:388, total_failure: 255, total_equal:20


---- round:663 --------
total_win:389, total_failure: 255, total_equal:20


---- round:664 --------
total_win:389, total_failure: 256, total_equal:20


---- round:665 --------
total_win:390, total_failure: 256, total_equal:20


---- round:666 --------
total_win:391, total_failure: 256, total_equal:20


---- round:667 --------
total_win:391, total_failure: 256, total_equal:21


---- round:668 --------
total_win:392, total_failure: 256, total_equal:21


---- round:669 --------
total_win:393, total_failure: 256, total_equal:21


---- round:670 --------
total_win:393, total_failure: 257, total_equal:21


---- round:671 --------
total_win:394, total_failure: 257, total_equal:21


---- round:672 --------
total_win:394, total_failure: 258, total_equal:21


---- round:673 --------
total_win:394, total_failure: 259, total_equal:21


---- round:


---- round:769 --------
total_win:462, total_failure: 284, total_equal:24


---- round:770 --------
total_win:463, total_failure: 284, total_equal:24


---- round:771 --------
total_win:464, total_failure: 284, total_equal:24


---- round:772 --------
total_win:464, total_failure: 284, total_equal:25


---- round:773 --------
total_win:465, total_failure: 284, total_equal:25


---- round:774 --------
total_win:465, total_failure: 285, total_equal:25


---- round:775 --------
total_win:466, total_failure: 285, total_equal:25


---- round:776 --------
total_win:466, total_failure: 286, total_equal:25


---- round:777 --------
total_win:467, total_failure: 286, total_equal:25


---- round:778 --------
total_win:468, total_failure: 286, total_equal:25


---- round:779 --------
total_win:468, total_failure: 287, total_equal:25


---- round:780 --------
total_win:468, total_failure: 288, total_equal:25


---- round:781 --------
total_win:469, total_failure: 288, total_equal:25


---- round:


---- round:878 --------
total_win:525, total_failure: 323, total_equal:31


---- round:879 --------
total_win:526, total_failure: 323, total_equal:31


---- round:880 --------
total_win:527, total_failure: 323, total_equal:31


---- round:881 --------
total_win:528, total_failure: 323, total_equal:31


---- round:882 --------
total_win:529, total_failure: 323, total_equal:31


---- round:883 --------
total_win:529, total_failure: 324, total_equal:31


---- round:884 --------
total_win:530, total_failure: 324, total_equal:31


---- round:885 --------
total_win:530, total_failure: 324, total_equal:32


---- round:886 --------
total_win:530, total_failure: 325, total_equal:32


---- round:887 --------
total_win:531, total_failure: 325, total_equal:32


---- round:888 --------
total_win:532, total_failure: 325, total_equal:32


---- round:889 --------
total_win:533, total_failure: 325, total_equal:32


---- round:890 --------
total_win:534, total_failure: 325, total_equal:32


---- round:


---- round:987 --------
total_win:591, total_failure: 362, total_equal:35


---- round:988 --------
total_win:592, total_failure: 362, total_equal:35


---- round:989 --------
total_win:592, total_failure: 362, total_equal:36


---- round:990 --------
total_win:593, total_failure: 362, total_equal:36


---- round:991 --------
total_win:594, total_failure: 362, total_equal:36


---- round:992 --------
total_win:595, total_failure: 362, total_equal:36


---- round:993 --------
total_win:596, total_failure: 362, total_equal:36


---- round:994 --------
total_win:596, total_failure: 363, total_equal:36


---- round:995 --------
total_win:597, total_failure: 363, total_equal:36


---- round:996 --------
total_win:598, total_failure: 363, total_equal:36


---- round:997 --------
total_win:598, total_failure: 364, total_equal:36


---- round:998 --------
total_win:598, total_failure: 365, total_equal:36


---- round:999 --------
total_win:599, total_failure: 365, total_equal:36

train time: 

In [4]:
!python reversi_model_train.py --board_size 8 --total_timesteps 100000 --cp_timesteps 20000 --n_envs 8 --opponent_model_path random --start_index 0



load model from self.model_path: models/Reversi_ppo_8x8/model error
Using cuda device
Logging to models/Reversi_ppo_8x8/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.03     |
|    ep_rew_mean     | -1       |
| time/              |          |
|    fps             | 107      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 512      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.05         |
|    ep_rew_mean          | -1           |
| time/                   |              |
|    fps                  | 149          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0073447656 |
|    clip_fraction        | 0.333        |
|    clip_range   

Traceback (most recent call last):
  File "reversi_model_train.py", line 225, in <module>
    run_train(sys.argv[1:])
  File "reversi_model_train.py", line 220, in run_train
    print(f"end time: {time2str(time.time())}")
NameError: name 'time2str' is not defined


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.32        |
|    ep_rew_mean          | -1          |
| time/                   |             |
|    fps                  | 234         |
|    iterations           | 9           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4608        |
| train/                  |             |
|    approx_kl            | 0.014122702 |
|    clip_fraction        | 0.53        |
|    clip_range           | 0.1         |
|    entropy_loss         | -3.8        |
|    explained_variance   | -27.1       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0721     |
|    n_updates            | 32          |
|    policy_gradient_loss | -0.027      |
|    value_loss           | 0.000412    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.39  

| train/                  |             |
|    approx_kl            | 0.016008945 |
|    clip_fraction        | 0.403       |
|    clip_range           | 0.1         |
|    entropy_loss         | -2.24       |
|    explained_variance   | -1.93       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0448     |
|    n_updates            | 144         |
|    policy_gradient_loss | -0.0205     |
|    value_loss           | 0.000332    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.56        |
|    ep_rew_mean          | -1          |
| time/                   |             |
|    fps                  | 277         |
|    iterations           | 38          |
|    time_elapsed         | 70          |
|    total_timesteps      | 19456       |
| train/                  |             |
|    approx_kl            | 0.011021307 |
|    clip_fraction        | 0.329 

|    learning_rate        | 0.00025     |
|    loss                 | -0.0489     |
|    n_updates            | 468         |
|    policy_gradient_loss | -0.0131     |
|    value_loss           | 0.00075     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.38        |
|    ep_rew_mean          | -1          |
| time/                   |             |
|    fps                  | 279         |
|    iterations           | 39          |
|    time_elapsed         | 71          |
|    total_timesteps      | 19968       |
| train/                  |             |
|    approx_kl            | 0.020007072 |
|    clip_fraction        | 0.277       |
|    clip_range           | 0.1         |
|    entropy_loss         | -1.01       |
|    explained_variance   | -0.572      |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0403     |
|    n_updates            | 472   

|    explained_variance   | -0.278      |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0276     |
|    n_updates            | 612         |
|    policy_gradient_loss | -0.00273    |
|    value_loss           | 0.000403    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.01        |
|    ep_rew_mean          | -1          |
| time/                   |             |
|    fps                  | 282         |
|    iterations           | 35          |
|    time_elapsed         | 63          |
|    total_timesteps      | 17920       |
| train/                  |             |
|    approx_kl            | 0.032047465 |
|    clip_fraction        | 0.268       |
|    clip_range           | 0.1         |
|    entropy_loss         | -0.869      |
|    explained_variance   | 0.0643      |
|    learning_rate        | 0.00025     |
|    loss                 | -0.053

### 模型加载

In [7]:
import numpy as np
from gym_reversi import ReversiEnv

def get_possible_actions(board, player_color):
    actions = []
    d = board.shape[-1]
    opponent_color = 1 - player_color
    for pos_x in range(d):
        for pos_y in range(d):
            if board[0, pos_x, pos_y] or board[1, pos_x, pos_y]:
                continue
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx = pos_x + dx
                    ny = pos_y + dy
                    n = 0
                    if nx not in range(d) or ny not in range(d):
                        continue
                    while board[opponent_color, nx, ny] == 1:
                        tmp_nx = nx + dx
                        tmp_ny = ny + dy
                        if tmp_nx not in range(d) or tmp_ny not in range(d):
                            break
                        n += 1
                        nx += dx
                        ny += dy
                    if n > 0 and board[player_color, nx, ny] == 1:
                        action = pos_x * d + pos_y
                        if action not in actions:
                            actions.append(action)
    return actions

def set_possible_actions_place(board, possible_actions, channel_index=2):
    board[channel_index, :, :] = 0
    # possible_actions = ReversiEnv.get_possible_actions(board, player_color)
    possible_actions_coords = [ReversiEnv.action_to_coordinate(board, _action) for _action in possible_actions]
    for pos_x, pos_y in possible_actions_coords:
        board[channel_index, pos_x, pos_y] = 1
    return board

def get_test_observation(board_size=4, player_color=0):
    # init board setting
    N_CHANNELS = 4
    # channels： 0: 黑棋位置， 1: 白棋位置， 2: 当前可合法落子位置，3：player 颜色
    observation = np.zeros((N_CHANNELS, board_size, board_size), dtype=int)

    observation[3, :, :] = player_color

    centerL = int(board_size / 2 - 1)
    centerR = int(board_size / 2)
    # self.observation[2, :, :] = 1
    # self.observation[2, (centerL) : (centerR + 1), (centerL) : (centerR + 1)] = 0
    observation[0, centerR, centerL] = 1
    observation[0, centerL, centerR] = 1
    observation[1, centerL, centerL] = 1
    observation[1, centerR, centerR] = 1
    possible_actions = get_possible_actions(observation, player_color)

    # 设置主玩家合法位置
    set_possible_actions_place(observation, possible_actions)

    return observation


def action_to_coordinate(board, action):
    return action // board.shape[-1], action % board.shape[-1]


def valid_action_mask(board_size=8, possible_actions=[]):
    valid_actions = np.zeros((board_size**2, ), dtype=np.uint8)
    for idx in possible_actions:
        valid_actions[idx] = 1
    return valid_actions


In [12]:
import torch
from stable_baselines3 import PPO
from sb3_contrib.ppo_mask import MaskablePPO

# PolicyModel = PPO
PolicyModel = MaskablePPO

model_path = "models/ppo_8x8_backbone/model_40w"
# model_path = "models/model_460w"
state_dict_path = model_path + '_state_dict.pt'

sb3_model = PolicyModel.load(model_path)

torch.save(sb3_model.policy.state_dict(), state_dict_path)


In [13]:
sb3_model.policy

MaskableActorCriticPolicy(
  (features_extractor): CustomCNN(
    (net): MyResNet(
      (resnet_block): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (downsample): Sequential(
            (0): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [5]:
sb3_model.policy.state_dict().keys()

odict_keys(['features_extractor.net.resnet_block.0.conv1.weight', 'features_extractor.net.resnet_block.0.bn1.weight', 'features_extractor.net.resnet_block.0.bn1.bias', 'features_extractor.net.resnet_block.0.bn1.running_mean', 'features_extractor.net.resnet_block.0.bn1.running_var', 'features_extractor.net.resnet_block.0.bn1.num_batches_tracked', 'features_extractor.net.resnet_block.0.conv2.weight', 'features_extractor.net.resnet_block.0.bn2.weight', 'features_extractor.net.resnet_block.0.bn2.bias', 'features_extractor.net.resnet_block.0.bn2.running_mean', 'features_extractor.net.resnet_block.0.bn2.running_var', 'features_extractor.net.resnet_block.0.bn2.num_batches_tracked', 'features_extractor.net.resnet_block.0.downsample.0.weight', 'features_extractor.net.resnet_block.0.downsample.1.weight', 'features_extractor.net.resnet_block.0.downsample.1.bias', 'features_extractor.net.resnet_block.0.downsample.1.running_mean', 'features_extractor.net.resnet_block.0.downsample.1.running_var', 'f

In [45]:
model.policy.action_net

Linear(in_features=256, out_features=16, bias=True)

In [1]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy

def build_model(state_dict_path, features_extractor_kwargs, observation_space=(4,8,8)):

    # features_extractor_kwargs=dict(features_dim=1024,
    #                                net_arch=[64, 128, 256],
    #                                # net_arch=[64, 128, 128],
    #                                # net_arch=[32, 64, 128],
    #                                kernel_size=3,
    #                                stride=1,
    #                                padding='same',
    #                                is_batch_norm=False),

    action_space = [observation_space[1]*observation_space[2]]

    policy_model = MaskableActorCriticPolicy(
                    observation_space = observation_space,
                    action_space = action_space,
                    net_arch=[256, 256],
                    features_extractor_class = CustomCNN,
                    features_extractor_kwargs = features_extractor_kwargs,
                    normalize_images= False,
    )

    return policy_model


In [8]:
import torch

# ppo_model = PPO.load("models/Reversi_ppo_4x4/model_100w")

# ## 保存模型
# torch.save(ppo_model.policy, 'models/Reversi_ppo_4x4/model_100w.pth')

# ## 读取模型
# pth_model = torch.load('models/Reversi_ppo_4x4/model_100w.pth')

observation = get_test_observation(board_size=8, player_color=0)
# observation

possible_actions = get_possible_actions(observation, player_color=0)

action_mask = valid_action_mask(board_size=8, possible_actions=possible_actions)


In [9]:

possible_actions

[19, 26, 37, 44]

In [10]:
action_mask

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [14]:
action, _states = sb3_model.predict(observation, deterministic=True, action_mask=action_mask)
action
# action_to_coordinate(observation, action)

TypeError: predict() got an unexpected keyword argument 'action_mask'

In [9]:
action

array(26, dtype=int64)

In [4]:
policy_model

MaskableActorCriticPolicy(
  (features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): ReLU()
        (6): Flatten(start_dim=1, end_dim=-1)
      )
    )
    (linear): Sequential(
      (0): Linear(in_features=8192, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): ReLU()
        (6): Flat

In [1]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy


backbone='cnn'
backbone='res_net'

features_extractor_kwargs=dict(features_dim=256,
                               backbone=backbone,
#                                net_arch=[64, 128, 256],
                               net_arch=[64, 128, 128],
#                                net_arch=[32, 64, 128],
#                                net_arch=[32, 64, 64],
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               is_batch_norm=False
                               )

observation_space=(4, 8, 8)
action_space = [observation_space[1]*observation_space[2]]

policy_model = MaskableActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)

# model_path = "models/ppo_8x8_cnn/model"
model_path = "models/ppo_8x8_backbone/model_40w"
# model_path = "models/model_460w"

# model_path = "models/model"
state_dict_path = model_path + '_state_dict.pt'

policy_model.load_state_dict(torch.load(state_dict_path))

<All keys matched successfully>

In [5]:
action_masks = 
action, _states = policy_model.predict(observation, deterministic=True, action_masks)
action

array([0], dtype=int64)

In [35]:
action.reshape(1)

array([37], dtype=int64)

In [38]:
int(action)

37

In [39]:
import torch
from stable_baselines3 import PPO
PolicyModel = PPO

model_path = "models/ppo_8x8_cnn/model_1480w"
# model_path = "models/model_460w"
state_dict_path = model_path + '_state_dict.pt'

sb3_model = PolicyModel.load(model_path)

torch.save(sb3_model.policy.state_dict(), state_dict_path)

In [40]:
# model_path = "models/ppo_8x8_cnn/model"
model_path = "models/ppo_8x8_cnn/model_1480w"
# model_path = "models/model_460w"

# model_path = "models/model"
state_dict_path = model_path + '_state_dict.pt'

# features_extractor_kwargs=dict(features_dim=1024,
#                                net_arch=[64, 128, 256],
#                                # net_arch=[64, 128, 128],
#                                # net_arch=[32, 64, 128],
#                                kernel_size=3,
#                                stride=1,
#                                padding='same',
#                                is_batch_norm=False),

features_extractor_kwargs=dict(features_dim=256,
#                                net_arch=[64, 128, 256],
#                                net_arch=[64, 128, 128],
                               net_arch=[32, 64, 64],
                               kernel_size=3,
                               stride=1,
                               padding='same',
                               is_batch_norm=False)

policy_model = build_model(state_dict_path, features_extractor_kwargs, observation_space=(4,8,8))

policy_model.load_state_dict(torch.load(state_dict_path))
action, _states = policy_model.predict(observation, deterministic=True)
action

array([[44]], dtype=int64)

In [29]:
action, _states = policy_model.predict(observation, deterministic=True)
action

array([[37]], dtype=int64)

In [10]:
sb3_model.policy

ActorCriticPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=Tr

In [9]:
policy_model

ActorCriticPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=4096, out_features=256, bias=Tr

In [30]:
policy_model.state_dict()

OrderedDict([('features_extractor.cnn.0.weight',
              tensor([[[[ 5.8862e-02,  3.7363e-01, -1.3097e-01],
                        [-1.3117e-01, -1.4281e-01,  1.0642e-02],
                        [-3.2974e-01,  5.1945e-02, -1.7016e-01]],
              
                       [[-1.6550e-01,  2.2424e-01,  6.2481e-01],
                        [ 1.1638e-01,  6.0274e-02,  2.3141e-01],
                        [-1.1957e-01, -3.1702e-01,  2.2510e-01]],
              
                       [[-1.1080e-01,  3.4992e-01, -2.3739e-01],
                        [-8.8834e-02,  2.8504e-01,  1.4751e-01],
                        [ 1.0280e-01, -2.0961e-01, -1.3982e-01]],
              
                       [[-3.6928e-01,  2.0983e-01, -2.5590e-01],
                        [ 1.8024e-02,  4.0449e-01, -2.9480e-01],
                        [-1.0230e-01, -4.0876e-02,  4.6758e-01]]],
              
              
                      [[[ 4.3953e-02, -1.4663e-01,  3.7173e-01],
                        [ 

In [12]:
sb3_model.policy.state_dict()

OrderedDict([('features_extractor.cnn.0.weight',
              tensor([[[[ 5.8862e-02,  3.7363e-01, -1.3097e-01],
                        [-1.3117e-01, -1.4281e-01,  1.0642e-02],
                        [-3.2974e-01,  5.1945e-02, -1.7016e-01]],
              
                       [[-1.6550e-01,  2.2424e-01,  6.2481e-01],
                        [ 1.1638e-01,  6.0274e-02,  2.3141e-01],
                        [-1.1957e-01, -3.1702e-01,  2.2510e-01]],
              
                       [[-1.1080e-01,  3.4992e-01, -2.3739e-01],
                        [-8.8834e-02,  2.8504e-01,  1.4751e-01],
                        [ 1.0280e-01, -2.0961e-01, -1.3982e-01]],
              
                       [[-3.6928e-01,  2.0983e-01, -2.5590e-01],
                        [ 1.8024e-02,  4.0449e-01, -2.9480e-01],
                        [-1.0230e-01, -4.0876e-02,  4.6758e-01]]],
              
              
                      [[[ 4.3953e-02, -1.4663e-01,  3.7173e-01],
                        [ 